<a href="https://colab.research.google.com/github/kilchinskiy/Hw.3.ipynb/blob/main/Hw_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

df = pd.read_csv('/content/drive/MyDrive/Housing.csv')
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [49]:
# Обираємо ознаки (features), які будемо використовувати для прогнозу:
# area — площа будинку
# bathrooms — кількість ванних
# bedrooms — кількість спалень
X = df[['area','bathrooms','bedrooms']].values
X = df[['area','bathrooms','bedrooms']].values
# Обираємо цільову змінну (target) — ціну будинку
y = df['price'].values

# Додаємо bias колонку (стовпець одиниць)
# Це потрібно для навчання вільного коефіцієнта (intercept)
ones = np.ones((X.shape[0],1))
# Об'єднуємо bias та ознаки в одну матрицю
# Тепер перший стовпець — це 1
X = np.hstack((ones, X))
# Виводимо перші 5 рядків для перевірки
X[:5]

array([[1.00e+00, 7.42e+03, 2.00e+00, 4.00e+00],
       [1.00e+00, 8.96e+03, 4.00e+00, 4.00e+00],
       [1.00e+00, 9.96e+03, 2.00e+00, 3.00e+00],
       [1.00e+00, 7.50e+03, 2.00e+00, 4.00e+00],
       [1.00e+00, 7.42e+03, 1.00e+00, 4.00e+00]])

In [52]:
# Функція гіпотези лінійної регресії
# Обчислює прогнозовані значення y_hat

def hypothesis(X, w):
    return X @ w
    # X — матриця ознак
    # w — вектор ваг (коефіцієнтів)
    # @ — це матричне множення

In [53]:
# Функція втрат (Mean Squared Error)
# Показує наскільки модель помиляється
def loss_function(X, y, w):
    # Кількість прикладів у датасеті
    m = len(y)
    # Робимо прогноз за допомогою гіпотези
    pred = hypothesis(X, w)
    # Обчислюємо середню квадратичну помилку
    # (1/(2m)) використовується для зручності похідної
    return (1/(2*m)) * np.sum((pred - y)**2)


In [40]:
# Один крок градієнтного спуску
# Оновлює ваги w
def gradient_step(X, y, w, alpha):
    # Кількість прикладів
    m = len(y)
    # Прогноз моделі
    pred = hypothesis(X, w)
    # Обчислення градієнта
    # X.T — транспонована матриця ознак
    gradient = (1/m) * (X.T @ (pred - y))
    # Оновлення ваг
    # alpha — швидкість навчання (learning rate)
    w = w - alpha * gradient
    return w

In [54]:
# Беремо сирі ознаки без bias
X_raw = df[['area','bathrooms','bedrooms']].values
# Нормалізація потрібна щоб gradient descent працював стабільно
# Бо площа має великі значення

# Середнє значення кожної колонки
y = df['price'].values


X_mean = X_raw.mean(axis=0)
# Стандартне відхилення
X_std = X_raw.std(axis=0)
# Стандартизація (z-score)
X_norm = (X_raw - X_mean) / X_std
# Додаємо bias колонку
ones = np.ones((X_norm.shape[0],1))
# Об'єднуємо bias та нормалізовані дані
X = np.hstack((ones, X_norm))



In [56]:
# Функція повного градієнтного спуску
# Багаторазово виконує gradient_step

def train_gradient_descent(X, y, alpha=0.01, iterations=5000):
    # Ініціалізуємо ваги нулями
    w = np.zeros(X.shape[1])
    # Запускаємо цикл навчання
    for i in range(iterations):
        # Оновлюємо ваги
        w = gradient_step(X, y, w, alpha)
     # Повертаємо знайдені ваги
    return w
# Запускаємо навчання
w_gd = train_gradient_descent(X, y)
# Дивимось результат
w_gd

array([4766729.24770638,  821214.14349519,  695808.52272536,
        299983.57107964])

In [57]:
# Аналітичне рішення без ітерацій
# Використовує формулу нормального рівняння
def normal_equation(X, y):
    return np.linalg.inv(X.T @ X) @ X.T @ y
# Обчислюємо ваги
w_normal = normal_equation(X, y)
# Виводимо результат
w_normal

array([4766729.24770642,  821214.14349519,  695808.52272538,
        299983.57107963])

In [58]:
# Імпортуємо готову модель лінійної регресії
from sklearn.linear_model import LinearRegression

# Використовуємо ті ж нормалізовані ознаки
X_sklearn = X_norm
# Створюємо модель
model = LinearRegression()
# Навчаємо модель
model.fit(X_sklearn, y)
# Виводимо вільний член (intercept)
print("Intercept:", model.intercept_)
# Виводимо коефіцієнти
print("Coefficients:", model.coef_)


Intercept: 4766729.247706422
Coefficients: [821214.14349519 695808.52272537 299983.57107963]
